In [25]:
import cv2
import os
import numpy as np

#os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [26]:
base_dir = "./data" 
sub_dirs = [
    'train',
    'test'
]  
categories = [
    'Apple Scab Leaf',
    'Apple leaf',
    'Apple rust leaf',
    'Bell_pepper leaf',
    'Bell_pepper leaf spot',
    'Blueberry leaf',
    'Cherry leaf',
    'Corn Gray leaf spot',
    'Corn leaf blight',
    'Corn rust leaf',
    'Peach leaf',
    'Potato leaf early blight',
    'Potato leaf late blight',
    'Raspberry leaf',
    'Soyabean leaf',
    'Squash Powdery mildew leaf',
    'Strawberry leaf',
    'Tomato Early blight leaf',
    'Tomato Septoria leaf spot',
    'Tomato leaf',
    'Tomato leaf bacterial spot',
    'Tomato leaf late blight',
    'Tomato leaf mosaic virus',
    'Tomato leaf yellow virus',
    'Tomato mold leaf',
    'Tomato two spotted spider mites leaf',
    'grape leaf',
    'grape leaf black rot'
]

images = {
    'train': {},
    'test': {}
}

resize_dim = (224, 224)

In [27]:
for sub_dir in sub_dirs:
    for category in categories:
        category_path = os.path.join(base_dir, sub_dir, category)
        images[sub_dir][category] = []
        
        if os.path.exists(category_path):
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                img = cv2.imread(img_path)

                img = cv2.resize(img, resize_dim)
                # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                images[sub_dir][category].append(img)
        else:
            print(f"Category path does not exist: {category_path}")


Category path does not exist: ./data/test/Tomato two spotted spider mites leaf


In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize the data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Including validation split
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training, validation, and test data
train_generator = train_datagen.flow_from_directory(
    base_dir + '/train',
    target_size=(224, 224),  # Resizing images to match the model input
    batch_size=32,
    class_mode='categorical',  # Assuming a multi-class classification problem
    subset='training'  # Specify training data for the split
)

validation_generator = train_datagen.flow_from_directory(
    base_dir + '/train',  # Reusing the train directory but splitting for validation
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Specify validation data
)

test_generator = test_datagen.flow_from_directory(
    base_dir + '/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1885 images belonging to 28 classes.
Found 457 images belonging to 28 classes.
Found 236 images belonging to 27 classes.


In [29]:
test_tomato_leaf_images = images['test']['Tomato leaf']

print(f"Loaded {len(test_tomato_leaf_images)} Tomato leaf images from the Testing set.")

Loaded 8 Tomato leaf images from the Testing set.


In [30]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np

In [31]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)  # Dropout layer to reduce overfitting

# Add global average pooling layer to further reduce dimensionality
feature_extractor = base_model.output
feature_extractor = GlobalAveragePooling2D()(feature_extractor)

# This model outputs the extracted features
feature_extraction_model = Model(inputs=base_model.input, outputs=feature_extractor)

feature_extraction_model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [32]:
import cirq
import sympy
import tensorflow_quantum as tfq

featSize = 6

def create_quantum_model():
    # Define a small set of qubits
    qubits = [cirq.GridQubit(0, i) for i in range(featSize)]  # Adjust based on feature size

    # Build the quantum circuit
    circuit = cirq.Circuit()
    for qubit in qubits:
        circuit.append(cirq.H(qubit))  # Hadamard gate for superposition
        theta = sympy.Symbol(f"theta_{qubit}")
        circuit.append(cirq.rz(theta)(qubit))  # Rotation Z gate with parameter theta
        # Entangle qubits as needed
        if qubit != qubits[-1]:
            circuit.append(cirq.CNOT(qubit, qubits[qubits.index(qubit) + 1]))

    # Observable for the readout
    observable = cirq.Z(qubits[-1])  # Example observable

    return circuit, observable

# Quantum circuit and observable
qcircuit, observable = create_quantum_model()

# Quantum Keras layer
quantum_layer = tfq.layers.PQC(qcircuit, operators=observable)


In [33]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Setup a learning rate reduction strategy
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

# Classical feature extraction
inputs = Input(shape=(224, 224, 3))
extracted_features = feature_extraction_model(inputs)

# Placeholder for the processed features to be fed into the quantum model
# This step would involve encoding the features into quantum states
quantum_input = tf.keras.layers.Lambda(lambda x: tfq.convert_to_tensor([cirq.Circuit()]))(extracted_features)

# Quantum model processing
quantum_output = quantum_layer(quantum_input)

initial_learning_rate = 1e-4
optimizer = Adam(learning_rate=initial_learning_rate)

# Final model
model = Model(inputs=[inputs], outputs=[quantum_output])

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 model_5 (Functional)        (None, 1280)              2257984   
                                                                 
 lambda_2 (Lambda)           (1,)                      0         
                                                                 
 pqc_2 (PQC)                 (1, 1)                    6         
                                                                 
Total params: 2,257,990
Trainable params: 2,223,878
Non-trainable params: 34,112
_________________________________________________________________


In [34]:
# Assuming 'model' is your hybrid model prepared earlier
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Number of epochs to train for
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
58/58 [==============================] - 33s 540ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 2/10
58/58 [==============================] - 30s 495ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 3/10
58/58 [==============================] - 28s 489ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 4/10
58/58 [==============================] - 29s 493ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 5/10
58/58 [==============================] - 29s 496ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 6/10
58/58 [==============================] - 29s 507ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 7/10
58/58 [==============================] - 30s 498ms/step - loss: 1.1921e-07 - accura

In [35]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


8/8 [==============================] - 2s 215ms/step - loss: 1.1921e-07 - accuracy: 0.9630
Test Loss: 1.1920928955078125e-07
Test Accuracy: 0.9629630446434021
